In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, regexp_replace
from pyspark.sql.types import ArrayType, StructType, StructField, IntegerType, StringType

# stop any existing Spark session, if Spark is already running, creating a new session might fail ***
try:
    spark.stop()
except Exception:
    pass

# create session with adjusted memory settings based on your cluster
# .config("spark.local.dir", r"E:\Apache Spark\spark-temp"): change the spark local dir, as the c disk memory is not enough, may cause Py4JJavaError exception ***
spark = SparkSession.builder.appName("MovieRecommender_SQL_Movielens") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.local.dir", r"E:\Apache Spark\spark-temp") \
    .config("spark.jars", r"D:\MySQL\MySQL ConnectorJ\mysql-connector-j-8.4.0\mysql-connector-j-8.4.0\mysql-connector-j-8.4.0.jar") \
    .getOrCreate()

In [2]:
spark

### Read Movies from Database

In [3]:
# Read data from MySQL
movies = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3307/db_movie_recommender_sys?useSSL=false&serverTimezone=UTC") \
    .option("dbtable", "tb_movie") \
    .option("user", "root") \
    .option("password", "137162") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()

In [4]:
movies.show()

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
|       5|Father of the Bri...|              Comedy|
|       6|         Heat (1995)|Action|Crime|Thri...|
|       7|      Sabrina (1995)|      Comedy|Romance|
|       8| Tom and Huck (1995)|  Adventure|Children|
|       9| Sudden Death (1995)|              Action|
|      10|    GoldenEye (1995)|Action|Adventure|...|
|      11|American Presiden...|Comedy|Drama|Romance|
|      12|Dracula: Dead and...|       Comedy|Horror|
|      13|        Balto (1995)|Adventure|Animati...|
|      14|        Nixon (1995)|               Drama|
|      15|Cutthroat Island ...|Action|Adventure|...|
|      16|       Casino (1995)|         Crime|

In [5]:
print(movies.count())

27278


### Read Ratings from Database

In [6]:
ratings = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3307/db_movie_recommender_sys?useSSL=false&serverTimezone=UTC") \
    .option("dbtable", "tb_rating") \
    .option("user", "root") \
    .option("password", "137162") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()

In [7]:
ratings.show()

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|  78842|    4963|   1.5|2005-07-08 02:50:29|
|  78842|    4974|   4.5|2005-07-11 00:16:32|
|  78842|    5049|     2|2005-07-10 13:49:57|
|  78842|    5292|     5|2005-07-10 23:10:07|
|  78842|    5378|     4|2005-07-10 14:54:20|
|  43683|    1298|     3|2010-06-19 16:03:30|
|  78842|    5464|     3|2005-07-08 02:49:11|
|  78842|    5500|   2.5|2005-07-08 03:00:12|
|  43683|    1348|   4.5|2011-02-27 17:40:06|
|  43683|    1441|     4|2010-06-19 16:03:21|
|  78842|    5669|     3|2005-07-10 02:29:17|
|  43683|    1673|   2.5|2010-07-27 21:46:55|
|  78842|    5900|     2|2005-07-11 00:16:43|
|  43683|    1753|   2.5|2010-06-19 16:22:58|
|  78842|    5970|   0.5|2005-07-10 23:37:18|
|  43683|    1952|   3.5|2010-06-19 00:32:33|
|  78842|    6223|   3.5|2005-07-11 00:13:19|
|  43683|    2137|   2.5|2011-05-21 18:40:19|
|  78842|    6298|     2|2005-07-1

In [8]:
print(ratings.count())

2501155


In [9]:
from pyspark.sql.functions import col  # the count is every user's rating counting

rating_count = ratings.groupBy("user_id") \
       .count() \
       .orderBy(col("count"), ascending=False)

rating_count.show()

+-------+-----+
|user_id|count|
+-------+-----+
| 118205| 1198|
|   8405|  878|
| 125794|  709|
|  74142|  691|
| 121535|  669|
|  34576|  649|
|  82418|  649|
|  83090|  640|
| 131904|  632|
|  59477|  622|
|   8963|  599|
| 130767|  575|
|  15617|  571|
|  92011|  555|
|  79159|  547|
|  46470|  546|
| 120575|  534|
|  20132|  494|
|  71975|  490|
|  18138|  489|
+-------+-----+
only showing top 20 rows



In [10]:
rating_count_nums_row = rating_count.count()
print(rating_count_nums_row)

136972


In [11]:
rating_count = rating_count.filter(rating_count['count'] > 200)

In [12]:
rating_count_nums_row = rating_count.count()
print(rating_count_nums_row)

472


In [13]:
user_ids = rating_count.select("user_id")

user_id_list = [row['user_id'] for row in user_ids.collect()]  # get the user id list

In [14]:
user_id_list

['118205',
 '8405',
 '125794',
 '74142',
 '121535',
 '34576',
 '82418',
 '83090',
 '131904',
 '59477',
 '8963',
 '130767',
 '15617',
 '92011',
 '79159',
 '46470',
 '120575',
 '20132',
 '71975',
 '18138',
 '130459',
 '18611',
 '9544',
 '92269',
 '31122',
 '125978',
 '88820',
 '91193',
 '63147',
 '111549',
 '35128',
 '54465',
 '24688',
 '114406',
 '68026',
 '51703',
 '53346',
 '41267',
 '70201',
 '105580',
 '136268',
 '14705',
 '129583',
 '7201',
 '123606',
 '61168',
 '67346',
 '107326',
 '79531',
 '131347',
 '91867',
 '68063',
 '27469',
 '119048',
 '24219',
 '26867',
 '12131',
 '86529',
 '113668',
 '62812',
 '92956',
 '107640',
 '131894',
 '128309',
 '76630',
 '50297',
 '80920',
 '33736',
 '59414',
 '131961',
 '32344',
 '51558',
 '22901',
 '80092',
 '42929',
 '43194',
 '72008',
 '116317',
 '27053',
 '4222',
 '128258',
 '2261',
 '116189',
 '106441',
 '97853',
 '133811',
 '3907',
 '75299',
 '52636',
 '66763',
 '52009',
 '31404',
 '26193',
 '117144',
 '49554',
 '119531',
 '73611',
 '122995

In [15]:
# filters the ratings DataFrame to include only rows with user IDs that are in y—that is, 
# only ratings by users with more than 200 ratings.
ratings = ratings.filter(ratings.user_id.isin(user_id_list))

In [16]:
ratings.show()

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
| 113668|      29|   4.5|2004-06-07 03:20:09|
| 113668|      42|   1.5|2006-04-22 01:03:38|
| 113668|      46|     4|1999-01-13 05:05:06|
| 113668|      63|   3.5|2004-12-12 03:58:12|
| 113668|      70|     4|1999-01-13 21:49:52|
| 113668|      71|   0.5|2006-03-21 19:45:28|
| 113668|      81|   4.5|2004-06-30 00:50:50|
| 113668|      88|   1.5|2004-04-11 03:13:59|
| 113668|      97|     4|1999-03-21 23:41:43|
| 113668|     151|     2|2005-01-17 03:37:17|
| 113668|     160|   2.5|2003-12-22 17:02:57|
| 113668|     170|     4|1999-01-13 22:00:13|
| 113668|     175|   4.5|2004-12-30 04:35:41|
| 113668|     180|   4.5|2004-06-30 01:18:15|
| 113668|     196|     3|2000-12-20 21:50:40|
| 113668|     204|   0.5|2006-03-15 21:31:15|
| 113668|     228|   1.5|2003-12-07 19:11:20|
| 113668|     239|     1|2006-03-18 06:43:36|
| 113668|     289|   4.5|2003-05-2

In [17]:
print(ratings.count())

135483


### Integrate Movies and Ratings Data and Analyze them

In [18]:
# join the ratings with the movies
ratings_with_movies = ratings.join(movies, on="movie_id", how="inner")

In [19]:
ratings_with_movies.show()

+--------+-------+------+-------------------+--------------------+--------------------+
|movie_id|user_id|rating|          timestamp|               title|              genres|
+--------+-------+------+-------------------+--------------------+--------------------+
|    6240| 113668|     1|2003-10-27 05:00:04| One Good Cop (1991)|  Action|Crime|Drama|
|    2294|  26193|   3.5|2005-08-30 19:03:37|         Antz (1998)|Adventure|Animati...|
|   88140|  26193|   3.5|2014-12-25 04:30:33|Captain America: ...|Action|Adventure|...|
|    2069|   8963|   3.5|2004-12-24 15:38:32|Trip to Bountiful...|               Drama|
|    5325|   8963|     3|2002-05-16 02:36:53|Dogtown and Z-Boy...|         Documentary|
|    5645| 130767|     3|2008-09-21 22:35:19|   Texasville (1990)|               Drama|
|    2162|  52636|     4|2001-10-04 17:30:13|NeverEnding Story...|Adventure|Childre...|
|    2294|  52636|     4|2001-10-20 15:54:30|         Antz (1998)|Adventure|Animati...|
|    3959|    156|     5|2002-12

In [20]:
ratings_with_movies = ratings_with_movies.dropDuplicates(['movie_id', 'user_id']) # this is quite important, each user can rate the same movie multiple times

In [21]:
# The agg() function in this code is used to apply one or more aggregate functions to a DataFrame after grouping it by a specific column (in this case, "title").
from pyspark.sql.functions import count

rating_count_every_title = ratings_with_movies.groupBy('title').agg(count('rating').alias('rating_count'))

In [22]:
rating_count_every_title.show()

+--------------------+------------+
|               title|rating_count|
+--------------------+------------+
|When We Were King...|          20|
| If Lucy Fell (1996)|           7|
|Men in Black (a.k...|          51|
| Three Wishes (1995)|           7|
|    Elizabeth (1998)|          35|
|First Blood (Ramb...|          36|
|Goodbye, Mr. Chip...|           5|
|Boy Meets Girl (1...|           1|
|Blind Shaft (Mang...|           2|
|Problem Child (1990)|          22|
|7th Voyage of Sin...|          18|
|Germany Year Zero...|           5|
|Man Bites Dog (C'...|          16|
|Major Barbara (1941)|           1|
|One False Move (1...|          17|
|       Quills (2000)|          31|
|Before Night Fall...|          12|
|   Love Story (1970)|          26|
|  9th Company (2005)|           2|
|Paris, France (1993)|           3|
+--------------------+------------+
only showing top 20 rows



In [23]:
ratings_with_movies = ratings_with_movies.join(rating_count_every_title, on='title', how='inner')

In [24]:
ratings_with_movies = ratings_with_movies.filter(ratings_with_movies['rating_count'] >= 50)

In [25]:
ratings_with_movies.show()

+--------------------+--------+-------+------+-------------------+--------------------+------------+
|               title|movie_id|user_id|rating|          timestamp|              genres|rating_count|
+--------------------+--------+-------+------+-------------------+--------------------+------------+
|2001: A Space Ody...|     924| 101127|     4|2001-04-18 23:37:19|Adventure|Drama|S...|          56|
|2001: A Space Ody...|     924| 102025|     5|2002-12-08 10:44:22|Adventure|Drama|S...|          56|
|2001: A Space Ody...|     924|  10303|     5|2000-01-05 20:13:09|Adventure|Drama|S...|          56|
|2001: A Space Ody...|     924| 107356|     5|2002-06-14 00:46:00|Adventure|Drama|S...|          56|
|2001: A Space Ody...|     924| 108248|     3|2012-02-20 12:52:33|Adventure|Drama|S...|          56|
|2001: A Space Ody...|     924| 110190|   4.5|2011-08-09 05:13:20|Adventure|Drama|S...|          56|
|2001: A Space Ody...|     924|  11081|   3.5|2007-10-30 06:46:47|Adventure|Drama|S...|    

In [26]:
user_id_count = ratings_with_movies.groupBy('user_id').count().orderBy("count", ascending=False).show()


+-------+-----+
|user_id|count|
+-------+-----+
|  56504|   41|
|  53346|   40|
|  24688|   39|
|  68358|   39|
|  76630|   39|
|   5843|   38|
|  59873|   38|
|  77884|   37|
| 113991|   37|
|  61168|   37|
| 118754|   36|
|  94259|   36|
|  91893|   36|
|   4507|   36|
| 123606|   36|
|  54465|   36|
|  68063|   35|
|  32984|   35|
|  89307|   35|
|  51507|   35|
+-------+-----+
only showing top 20 rows



### Convert the final analyzed Datas into Pivot Table

In [27]:
spark.conf.set("spark.sql.pivotMaxValues", 20000)  # Adjust based on your data

In [28]:
# Get the pivot table of final data "ratings"
from pyspark.sql.functions import avg

# Pivot the table with user_id as columns, title as rows, and average rating as values
movie_pivot = ratings_with_movies.groupBy("title") \
                         .pivot("user_id") \
                         .agg(avg("rating")) \
                         .fillna(0)  # Optional: Replace nulls with 0


In [29]:
movie_pivot.show()

+--------------------+------+------+------+------+------+------+------+------+------+------+------+------+-----+-----+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+-----+------+------+------+------+-----+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+-----+------+------+------+------

In [30]:
from pyspark.sql.functions import monotonically_increasing_id

# Add an "index" column using monotonically_increasing_id()
movie_pivot = movie_pivot.withColumn("index", monotonically_increasing_id())

movie_pivot_indexed = movie_pivot.select("index", *[col for col in movie_pivot.columns if col != "index"]) # Reorder columns: Move "index" to the first column

In [31]:
movie_pivot_indexed.show()

+-----+--------------------+------+------+------+------+------+------+------+------+------+------+------+------+-----+-----+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+-----+------+------+------+------+-----+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+-----+------+------+------+

In [32]:
movie_pivot_indexed_collected = movie_pivot_indexed.collect()

In [ ]:
import pickle

movie_data_row = [row.asDict() for row in movie_pivot_indexed_collected] # Convert it to a list of dictionaries for easier handling

# save the movie_pivot_indexed_collected  to a .pkl file
with open("movie_pivot_indexed_collected.pkl", "wb") as f:
    pickle.dump(movie_data_row, f)

### Convert the movie_pivot into Sparse Matrix

### Using CoordinateMatrix, MatrixEntry
there are always some problems here, maybe latter

In [34]:
# from pyspark.sql.functions import monotonically_increasing_id
# from pyspark.sql.types import StructType, StructField, LongType, IntegerType, DoubleType
# from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

# # Add a row index column to the pivot table.
# movie_pivot_indexed = movie_pivot.withColumn("row_index", monotonically_increasing_id())

# movie_pivot_indexed.show()

# # Define the rating columns (all columns except 'title' and the new 'row_index')
# rating_columns = movie_pivot.columns[1:]

# # Convert each row into MatrixEntry objects.
# entries = movie_pivot_indexed.rdd.flatMap(lambda row: [
#     MatrixEntry(row.row_index, j, float(row[col]))
#     for j, col in enumerate(rating_columns)
#     if float(row[col]) != 0.0  # only include nonzero ratings
# ])

# movie_pivot_indexed.cache()

# # Explicitly convert each MatrixEntry to a tuple (i, j, value)
# entries_tuples = entries.map(lambda me: (me.i, me.j, me.value))

# # Define a schema matching MatrixEntry's fields.
# schema = StructType([
#     StructField("i", LongType(), False),
#     StructField("j", IntegerType(), False),
#     StructField("value", DoubleType(), False)
# ])

# # Create a DataFrame using the schema.
# entries_df = spark.createDataFrame(entries_tuples, schema=schema)

### Using the traditional way
Make sure you data row is lower than 250 0000, or the computer will crushed

In [ ]:
# why use Spark and its pyspark.sql.DataFrame instead of directly using Pandas? That way, we wouldn't need to convert it to a Pandas DataFrame in the code below ***
import numpy as np
from scipy.sparse import csr_matrix

movie_pivot_pandas = movie_pivot_indexed.toPandas()

# remove the non-numeric 'title' column
# (Alternatively, if 'title' is the index, you could reset the index or extract it separately)
numeric_matrix = movie_pivot_pandas.drop(columns=["index", "title"]).values.astype(np.float64)

# now create the sparse matrix
movie_sparse = csr_matrix(numeric_matrix)

In [36]:
type(movie_sparse)

scipy.sparse._csr.csr_matrix

In [ ]:
with open("movie_sparse.pkl", "wb") as f: # save the sparse matrix to a .pkl file
    pickle.dump(movie_sparse, f)

### Using KNearestNeighbors Algorithmn to achieve the Model

In [37]:
# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [38]:
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute')

In [39]:
movie_pivot_indexed.filter(movie_pivot_indexed.title == "Annie Hall (1977)").show()

+-----+-----------------+------+------+------+------+------+------+------+------+------+------+------+------+-----+-----+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+-----+------+------+------+------+-----+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+-----+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+-----+------+------+------+---

In [40]:
suggested_book_index = 107
movie_pivot_indexed_collected[suggested_book_index]["title"]

'Natural Born Killers (1994)'

In [41]:
# reshape(1, -1) ensures that the row is treated as a single data point (as a 2D array with one row and multiple columns). 
# reshape(1, -1) function is used to convert this 1D array into a 2D array with one row and multiple columns (features).
# for example: [1, 2, 3, 4, 5] --> [[1, 2, 3, 4, 5]]

row = movie_pivot_indexed_collected[suggested_book_index] # this gets suggested_book_index th row from the DataFrame

row_values = np.array(row[2:]) # the first col is index, the second row is title, we don't need it

distance, suggestion = model.kneighbors(row_values.reshape(1,-1), n_neighbors=6 )


In [42]:
distance

array([[ 0.        , 29.60152023, 30.30676492, 31.02015474, 31.08858311,
        31.30894441]])

In [43]:
suggestion

array([[107,  13, 156,  10, 118, 207]])

In [44]:
for i in range(len(suggestion[0])):
    index = int(suggestion[0][i]) # convert NumPy array value to integer
    print(movie_pivot_indexed_collected[index]['title'])

Natural Born Killers (1994)
Batman & Robin (1997)
Casper (1995)
Pearl Harbor (2001)
Honey, I Shrunk the Kids (1989)
Mr. & Mrs. Smith (2005)
